<a href="https://colab.research.google.com/github/ckj18/ToBigs_workspace/blob/main/week7_CNNbasic_AlexNet_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNbasic Assignment#2

# AlexNet 구현

모델 구현 후 summary로 전체 모델 구조 출력과 주석으로 간단한 설명을 달아주시면 됩니다.

프레임워크는 자유이고, 기본 tensforflow와 pytorch tutorial 사이트를 아래에 첨부해 드립니다.

이 외 각 프레임워크 별 summary 등 추가적인 사용 방법은 구글링으로 찾아주세요!-!

- Tensorflow Tutorial: https://www.tensorflow.org/tutorials?hl=ko

- Pytorch Tutorial: https://tutorials.pytorch.kr/

## Tensorflow(keras)

In [4]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential


n_classes = 1000 # class 개수

model=Sequential()

############## Add Layer ##############
model.add(Conv2D(96, (11,11), strides=4, padding='valid', input_shape=(227, 227, 3)))
model.add(MaxPooling2D(pool_size=(3,3), strides=2))

model.add(Conv2D(256, (5,5), strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3,3), strides=2))

model.add(Conv2D(384, (3,3), strides=1, activation='relu', padding='same'))
model.add(Conv2D(384, (3,3), strides=1, activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), strides=1, activation='relu', padding='same'))

model.add(MaxPooling2D(pool_size=(3,3), strides=2))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))
#######################################

# keras summary
model.summary() # summary code 추가

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 13, 13, 384)       885120    
                                                                 
 conv2d_13 (Conv2D)          (None, 13, 13, 384)      

## Pytorch

In [6]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


class AlexNet(nn.Module):

    def __init__(self, n_classes):
        super(AlexNet, self).__init__()
        
        ############## Add Layer ##############
        self.n_classes = n_classes
        self.Conv1 = nn.Conv2d(in_channels=3, out_channels=96, 
                               kernel_size=(11,11), stride=4, padding=0)
        self.Conv2 = nn.Conv2d(in_channels=96, out_channels=256, 
                               kernel_size=(5,5), stride=1, padding=2)
        self.Conv3 = nn.Conv2d(in_channels=256, out_channels=384, 
                               kernel_size=(3,3), stride=1, padding=1)
        self.Conv4 = nn.Conv2d(in_channels=384, out_channels=384, 
                               kernel_size=(3,3), stride=1, padding=1)
        self.Conv5 = nn.Conv2d(in_channels=384, out_channels=256, 
                               kernel_size=(3,3), stride=1, padding=1)
        self.Pool = nn.MaxPool2d(kernel_size=3, stride=2)

            
        self.fc1 = nn.Linear(in_features=256*6*6, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=self.n_classes)
        #######################################

    def forward(self, x):
        
        ############## Add Layer ##############
        x = self.Pool(self.Conv1(x))
        x = self.Pool(F.relu(self.Conv2(x)))
        x = F.relu(self.Conv3(x))
        x = F.relu(self.Conv4(x))
        x = self.Pool(F.relu(self.Conv5(x)))
        
        x = x.view(-1, 256*6*6)
        
        nn.Dropout(0.5)
        x = F.relu(self.fc1(x))

        nn.Dropout(0.5)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        #######################################
        
        return x

# pytorch summary
model = AlexNet(1000)
summary(model, input_size=(3, 227, 227))# summary code 추가

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
         MaxPool2d-2           [-1, 96, 27, 27]               0
            Conv2d-3          [-1, 256, 27, 27]         614,656
         MaxPool2d-4          [-1, 256, 13, 13]               0
            Conv2d-5          [-1, 384, 13, 13]         885,120
            Conv2d-6          [-1, 384, 13, 13]       1,327,488
            Conv2d-7          [-1, 256, 13, 13]         884,992
         MaxPool2d-8            [-1, 256, 6, 6]               0
            Linear-9                 [-1, 4096]      37,752,832
           Linear-10                 [-1, 4096]      16,781,312
           Linear-11                 [-1, 1000]       4,097,000
Total params: 62,378,344
Trainable params: 62,378,344
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.59
Fo